In [1]:
import numpy as np
import pandas as pd 
#import matplotlib.pyplot as plt
#import seaborn as sns
pd.set_option('max_columns', 50)
####model libraries ####################
from sklearn.linear_model import Ridge, Lasso,SGDRegressor,LinearRegression
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import ExtraTreeRegressor,DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
#########################################
from sklearn.cross_decomposition import PLSRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error,explained_variance_score
from sklearn.preprocessing import MinMaxScaler

#### visulization #####
import plotly.graph_objects as go



In [2]:
#df = pd.ExcelFile('C:/Users/halil/Desktop/soalr_data.xlsx').parse('sheet 1')
weather_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="weather")
guneko_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="1000255-GUNEKO")

In [3]:
guneko_production = guneko_raw[["Date","Production"]]
guneko_gti = guneko_raw[["Date.1","GTI"]]


weather_guneko = weather_raw.loc[weather_raw.name ==1000255]
ali = pd.merge(guneko_production,weather_guneko,left_on="Date",right_on="date")
dataset = ali.drop(["name","date","lat","lon"],axis=1)

In [4]:
conversion = dataset["Date"].astype(str)

dataset["year"] = conversion.apply(lambda x:int(x[:4]))
dataset["month"] = conversion.apply(lambda x:int(x[5:7]))
dataset["day"] = conversion.apply(lambda x:int(x[8:10]))
dataset["hour"] = conversion.apply(lambda x:int(x[11:13]))
onehot_encoded = dataset.drop(["Date"],axis=1)

In [ ]:

def metrics(y_train1,y_train_pred1,y_test1,y_test_pred1):
    print("for train data, mean squared error is",mean_squared_error(y_train1, y_train_pred1))
    print("for train data, mean absolute error is:",mean_absolute_error(y_train1, y_train_pred1))
    print("for train data, mean absolute percentage error is",mean_absolute_percentage_error(y_train1, y_train_pred1),"\n")

    print("for test data, mean squared error is",mean_squared_error(y_test1, y_test_pred1))
    print("for test data, mean absolute error is:",mean_absolute_error(y_test1, y_test_pred1))
    print("for test data, mean absolute percentage error is",mean_absolute_percentage_error(y_test1, y_test_pred1),"\n")


In [5]:
#normalization in range (-1,1)
def normalize(df,column_list):
    for col in column_list:
        feature_range = (0,1)
        min_max_scaler = MinMaxScaler(feature_range=feature_range)

        df[col] = min_max_scaler.fit_transform(df[col].values.reshape(-1,1))

    return df

In [67]:
def cal_model_error(x_train,x_test,y_train,y_test,model_name):
    model = model_name() 
    model.fit(x_train,y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)

    return round(mean_squared_error(y_train, train_prediction),4), round(mean_squared_error(y_test, test_prediction),4)
    #return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)


In [ ]:
"""

def calculate_ridge(x_train,x_test,y_train,y_test):
    model = Ridge(alpha=1) 
    model.fit(x_train,y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)

    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)



def calculate_lasso(x_train,x_test,y_train,y_test):
    model = Lasso(alpha=1) 
    model.fit(x_train,y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)

    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

    
def calculate_linear_reg(x_train,x_test,y_train,y_test):

    model = LinearRegression()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_xgb(x_train,x_test,y_train,y_test):
    model = XGBRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_kneighbors(x_train,x_test,y_train,y_test):
    model = KNeighborsRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_decisiontree(x_train,x_test,y_train,y_test):
    model = DecisionTreeRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_randomforest(x_train,x_test,y_train,y_test):
    model = RandomForestRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)
"""


In [7]:
def make_data(dataframe):
    train = dataframe[:len(dataframe)- 24*5]
    test = dataframe[len(dataframe)- 24*5:]

    x_train = train.copy()
    x_test = test.copy()

    y_train = x_train.pop('Production')
    y_test = x_test.pop('Production')

    return x_train, x_test, y_train, y_test 

In [8]:
def get_year(whole_data,target_year):

    whole_data = whole_data.loc[whole_data.year== target_year]
    whole_data = whole_data.drop(["year"],axis=1)
    whole_data = whole_data.reset_index(drop=True)

    return whole_data

In [9]:
def get_months(target_data):
    months = []

    for i in range(1,13):
        month_data = target_data.loc[target_data["month"]==i].copy()
        aw1 = month_data.reset_index(drop=True)
        aw1 = aw1.drop(["month"],axis=1)
        months.append(aw1)

    return months

In [ ]:
#x_train, x_test, y_train, y_test = make_data(data_months[0])

In [10]:
#error calculate method with only one function , dynamically but can give error 

def calculate_errors(month_data):
    errors = []
    ridge,lasso,decisiontree,kneighbors,linear,randomforest,xgb,adaboost,neuralnetwork= 0,0,0,0,0,0,0,0,0
    model_list = [ridge,lasso,decisiontree,kneighbors,linear,randomforest,xgb,adaboost,neuralnetwork]

    models = [Ridge,Lasso,DecisionTreeRegressor,KNeighborsRegressor,LinearRegression,RandomForestRegressor,XGBRegressor,AdaBoostRegressor,MLPRegressor,MLPRegressor]
    
    for i in range(0,12):
        x_train, x_test, y_train, y_test = make_data(month_data[i])

        
        for m in range(0,9):
            model_list[m] = cal_model_error(x_train, x_test, y_train, y_test,models[m])

        errors.append({f"ridge":model_list[0],"lasso":model_list[1],"decisiontree":model_list[2],"kneighbors":model_list[3],
        "linear":model_list[4],"randomforest":model_list[5],"xgb":model_list[6],"adaboost":model_list[7],"neuralnetwork":model_list[8]})
    return errors

In [ ]:
#error calculation method that passing different functions for each model
"""def calculate_errors(month_data):
    errors = []
    for i in range(0,12):
        x_train, x_test, y_train, y_test = make_data(month_data[i])

        ridge = calculate_ridge(x_train, x_test, y_train, y_test)
        lasso = calculate_lasso(x_train, x_test, y_train, y_test)
        decisiontree = calculate_decisiontree(x_train, x_test, y_train, y_test)
        kneighbors = calculate_kneighbors(x_train, x_test, y_train, y_test)
        linear= calculate_linear_reg(x_train, x_test, y_train, y_test)
        randomforest = calculate_randomforest(x_train, x_test, y_train, y_test)
        xgb =calculate_xgb(x_train, x_test, y_train, y_test)

        errors.append({f"ridge":ridge,"lasso":lasso,"decisiontree":decisiontree,
                "kneighbors":kneighbors,"linear":linear,"randomforest":randomforest,"xgb":xgb})
    return errors"""

In [12]:
def dataframe_maker(errors):    
    aylar = ["January","February","March","April","May","June","July","August","September","October","November","December"]

    ridge_train, ridge_test = [] , []
    lasso_train, lasso_test= [], []
    decisiontree_train, decisiontree_test = [], []
    kneighbors_train, kneighbors_test = [], []
    linear_train, linear_test = [], []
    randomforest_train, randomforest_test = [], []
    xgb_train, xgb_test = [], []
    adaboost_train, adaboost_test = [], []
    neuralnetwork_train, neuralnetwork_test = [], []
    for i in errors:
        ridge_train.append(i["ridge"][0])
        ridge_test.append(i["ridge"][1])

        lasso_train.append(i["lasso"][0])
        lasso_test.append(i["lasso"][1])

        decisiontree_train.append(i["decisiontree"][0])
        decisiontree_test.append(i["decisiontree"][1])

        kneighbors_train.append(i["kneighbors"][0])
        kneighbors_test.append(i["kneighbors"][1])

        linear_train.append(i["linear"][0])
        linear_test.append(i["linear"][1])

        randomforest_train.append(i["randomforest"][0])
        randomforest_test.append(i["randomforest"][1])
        
        xgb_train.append(i["xgb"][0])
        xgb_test.append(i["xgb"][1])

        adaboost_train.append(i["adaboost"][0])
        adaboost_test.append(i["adaboost"][1])

        neuralnetwork_train.append(i["neuralnetwork"][0])
        neuralnetwork_test.append(i["neuralnetwork"][1])
        

    seri5 = pd.Series(aylar)
    train_frames = {"months":seri5,"ridge":ridge_train,"lasso":lasso_train,"decisiontree":decisiontree_train,"kneighbors":kneighbors_train,
                    "linear":linear_train,"randomforest":randomforest_train,"xgb": xgb_train,"adaboost":adaboost_train,"neuralnetwork": neuralnetwork_train}      

    test_frames = {"months":seri5,"ridge":ridge_test,"lasso":lasso_test,"decisiontree":decisiontree_test,"kneighbors":kneighbors_test,
                    "linear":linear_test,"randomforest":randomforest_test,"xgb": xgb_test,"adaboost":adaboost_test,"neuralnetwork":neuralnetwork_test}  

    train_dataframe = pd.DataFrame(train_frames)
    test_dataframe = pd.DataFrame(test_frames)
    return train_dataframe, test_dataframe    

In [87]:
def plot_resulst(train, test, model_name):
    
    fig = go.Figure(layout_yaxis_range=[0,0.05])
    fig.add_trace(go.Bar(x=test["months"],y=train[model_name],name=f'Train Errors', marker_color='indianred',text=train[model_name],textposition="outside"))

    fig.add_trace(go.Bar(x=test["months"], y=test[model_name], name=f'Test Errors',marker_color='lightsalmon',text=test[model_name],textposition="outside"))

    # Here we modify the tickangle of the xaxis, resulting in rotated labels.
    fig.update_layout(barmode='group', xaxis_tickangle=-45,title=f"{model_name} model error results")
    fig.show()

In [53]:
normalized = normalize(onehot_encoded,onehot_encoded.columns[0:11])
data_2021 = get_year(normalized,2021)


In [68]:
data_months = get_months(data_2021)
errors = calculate_errors(data_months)
train_errors,test_errors = dataframe_maker(errors)

In [73]:
train_errors

,months,ridge,lasso,decisiontree,kneighbors,linear,randomforest,xgb,adaboost,neuralnetwork
0,January,0.0088,0.0416,0.0,0.0021,0.0087,0.0007,0.0,0.0041,0.0092
1,February,0.0111,0.0599,0.0,0.0037,0.0107,0.0008,0.0,0.0063,0.0115
2,March,0.0097,0.0669,0.0,0.0042,0.0088,0.0008,0.0,0.0053,0.0088
3,April,0.0145,0.0960,0.0,0.0071,0.0134,0.0014,0.0,0.0096,0.0155
4,May,0.0095,0.1356,0.0,0.0031,0.0092,0.0006,0.0,0.0062,0.0098
5,June,0.0098,0.1388,0.0,0.0028,0.0092,0.0009,0.0,0.0055,0.0123
6,July,0.0171,0.1194,0.0,0.0034,0.0165,0.0012,0.0,0.0118,0.0165
7,August,0.0123,0.1172,0.0,0.0034,0.0119,0.0009,0.0,0.0076,0.0129
8,September,0.0085,0.1236,0.0,0.0015,0.0082,0.0004,0.0,0.0031,0.0122
9,October,0.0079,0.1119,0.0,0.0016,0.0077,0.0005,0.0,0.0024,0.0095


In [74]:
test_errors

,months,ridge,lasso,decisiontree,kneighbors,linear,randomforest,xgb,adaboost,neuralnetwork
0,January,0.0068,0.0297,0.0099,0.0204,0.0067,0.0057,0.0077,0.0062,0.0084
1,February,0.0063,0.1056,0.0018,0.0025,0.0062,0.0015,0.0017,0.0063,0.0160
2,March,0.0165,0.1167,0.0207,0.1065,0.0157,0.0155,0.0170,0.0205,0.0284
3,April,0.0218,0.1098,0.0262,0.0324,0.0243,0.0165,0.0180,0.0234,0.0242
4,May,0.0122,0.1257,0.0220,0.0153,0.0120,0.0114,0.0141,0.0129,0.0312
5,June,0.0196,0.1178,0.0244,0.0165,0.0187,0.0172,0.0165,0.0284,0.0220
6,July,0.0073,0.1327,0.0015,0.0027,0.0066,0.0015,0.0013,0.0143,0.0134
7,August,0.0058,0.1220,0.0119,0.0027,0.0062,0.0011,0.0030,0.0094,0.0100
8,September,0.0078,0.1299,0.0011,0.0028,0.0073,0.0013,0.0010,0.0044,0.0171
9,October,0.0175,0.0694,0.0200,0.0151,0.0592,0.0119,0.0117,0.0130,0.0145


In [88]:
plot_resulst(train_errors,test_errors,"ridge")

In [89]:
plot_resulst(train_errors,test_errors,"lasso")

In [90]:
plot_resulst(train_errors,test_errors,"decisiontree")

In [91]:
plot_resulst(train_errors,test_errors,"kneighbors")

In [92]:
plot_resulst(train_errors,test_errors,"linear")

In [93]:
plot_resulst(train_errors,test_errors,"randomforest")

In [94]:
plot_resulst(train_errors,test_errors,"xgb")

In [95]:
plot_resulst(train_errors,test_errors,"adaboost")

In [96]:
plot_resulst(train_errors,test_errors,"neuralnetwork")